In [1]:

from typing import Callable, Tuple, TypeVar, cast, get_origin, get_args, Union, Any, Optional, Dict, List
from components import CallChannel, ComponentStarter

from components.two_wheels import TwoWheelsV3
from components.logger import LoggerComponent
from components.controller.bluetooth_controller import BlueToothCarControlSPP_V2
from components.controller.bluetooth_SPP_server import start_bluetooth_server_v2
from components.gyroscope.gyroscope import AngularSpeedControlV3
from components.camera import Picamera2V2
from data_collection.data_collection import LoggerSet
from multiprocessing import Manager
import numpy as np
import datetime
import RPi.GPIO as GPIO
GPIO.setmode(GPIO.BOARD)

# from components import numpy_sample_setup
# r, w = numpy_sample_setup(['d', 'd'], [0, 0])

%load_ext autoreload
%autoreload 2

In [2]:
#loggerset = LoggerSet('../log/testsession'+str(datetime.datetime.now()), overwrite_ok=False)
loggerset = LoggerSet('../log/session'+str(datetime.datetime.now()), overwrite_ok=False)
manager = Manager()

In [3]:
logger_starter = ComponentStarter(
    LoggerComponent, 
    manager, 
    init_kwargs=dict(
        loggerset = loggerset
    ),
    loop_intervals={'step': 1/100},
)

two_wheel_starter = ComponentStarter(
    TwoWheelsV3, 
    manager, 
    init_kwargs=dict(
        left_pin = 33, 
        right_pin = 32, 
        dir_pins = (16, 18), 
        name='TwoWheelsV3'
    ),
    loop_intervals={'step': 1/100},
    instantiator = TwoWheelsV3.entry

)

angular_speed_control_starter = ComponentStarter(
    AngularSpeedControlV3, 
    manager, 
    init_kwargs=dict(
        i2c_address=0x68, 
        bus_num=1,
        name='AngularSpeedControlV3'
    ),
    loop_intervals={'step': 1/100},
    instantiator = AngularSpeedControlV3.entry
)

bluetooth_control_starter = ComponentStarter(
    BlueToothCarControlSPP_V2, 
    manager, 
    init_kwargs={},
    loop_intervals={'step': 1/100},
)

bt_ser_out, bt_ser = start_bluetooth_server_v2(manager)


picamera_starter = ComponentStarter(
    Picamera2V2, 
    manager, 
    init_kwargs=dict(
        resolution=(114, 64), 
        framerate=30,
        name='Picamera2V2'
    ),
    loop_intervals={'step': 1/30},
    sample_setup_kwargs=dict(default_values=[np.zeros((64, 114, 3), dtype=np.uint8)])
)

Making discoverable...


Starting Serial Port Profile...
NewConnection(/org/bluez/hci0/dev_C8_BD_4D_BA_4F_15, 88)


In [4]:

two_wheel_starter.register_outgoing_rpc(
    dict(log=logger_starter.incoming_rpcs['log'])
)

angular_speed_control_starter.register_outgoing_rpc(
    dict(log=logger_starter.incoming_rpcs['log'])
)


bluetooth_control_starter.register_outgoing_rpc(
    dict(log=logger_starter.incoming_rpcs['log'])
)

picamera_starter.register_outgoing_rpc(
    dict(
        log=logger_starter.incoming_rpcs['log'],
        setup_video_saver=logger_starter.incoming_rpcs['setup_video_saver'],
        save_video_frame=logger_starter.incoming_rpcs['save_video_frame']
        )
)

two_wheel_starter.register_incoming_samples(
    angular_speed_control_starter.outgoing_sample_readers
)

angular_speed_control_starter.register_incoming_samples(
    bluetooth_control_starter.outgoing_sample_readers
)


bluetooth_control_starter.register_incoming_samples(
    bt_ser_out
)

In [5]:
bluetooth_control_starter.start()
logger_starter.start()
two_wheel_starter.start()
angular_speed_control_starter.start()
picamera_starter.start()

/home/kawa/projects/car1/src/components/syncronisation/__init__.py:333: UserWarning: the real interval is >= ideal_interval
  warnings.warn("the real interval is >= ideal_interval")


[2:18:13.407265913] [16091]  INFO Camera camera_manager.cpp:313 libcamera v0.3.0+65-6ddd79b5-dirty (2024-06-24T11:13:49+10:00)
[2:18:13.464088807] [16166]  WARN RPiSdn sdn.cpp:40 Using legacy SDN tuning - please consider moving SDN inside rpi.denoise
[2:18:13.467051693] [16166]  INFO RPI vc4.cpp:446 Registered camera /base/soc/i2c0mux/i2c@1/ov5647@36 to Unicam device /dev/media1 and ISP device /dev/media2
[2:18:13.467345800] [16166]  INFO RPI pipeline_base.cpp:1104 Using configuration file '/usr/local/share/libcamera/pipeline/rpi/vc4/rpi_apps.yaml'
[2:18:13.477770510] [16091]  INFO Camera camera_manager.cpp:313 libcamera v0.3.0+65-6ddd79b5-dirty (2024-06-24T11:13:49+10:00)
[2:18:13.512939755] [16169]  WARN RPiSdn sdn.cpp:40 Using legacy SDN tuning - please consider moving SDN inside rpi.denoise
[2:18:13.516418337] [16169]  INFO RPI vc4.cpp:446 Registered camera /base/soc/i2c0mux/i2c@1/ov5647@36 to Unicam device /dev/media1 and ISP device /dev/media2
[2:18:13.516485521] [16169]  INFO RP

saving: BlueToothCarControlSPP_V2
saving: TwoWheelsV3
saving: Picamera2V2
saving: AngularSpeedControlV3
saving: BlueToothCarControlSPP_V2
saving: TwoWheelsV3
saving: Picamera2V2
saving: AngularSpeedControlV3
saving: BlueToothCarControlSPP_V2
saving: TwoWheelsV3
saving: Picamera2V2
saving: AngularSpeedControlV3
saving: BlueToothCarControlSPP_V2
saving: TwoWheelsV3
saving: Picamera2V2
saving: AngularSpeedControlV3
saving: BlueToothCarControlSPP_V2
saving: TwoWheelsV3
saving: Picamera2V2
saving: AngularSpeedControlV3
saving: BlueToothCarControlSPP_V2
saving: TwoWheelsV3
saving: Picamera2V2
saving: AngularSpeedControlV3


In [19]:
import plotly.express as px
px.imshow(picamera_starter.outgoing_samples[0]()/255)

In [13]:
img = picamera_starter.outgoing_samples[0]()/255
img.T.shape

(3, 114, 64)

In [6]:
bt_ser.terminate()
bluetooth_control_starter.process.terminate()
two_wheel_starter.process.terminate()
angular_speed_control_starter.process.terminate()
picamera_starter.process.terminate()
#logger_starter.process.terminate()


# playground

In [8]:
l = logger_starter.incoming_rpcs['get_logger'].call('TwoWheelsV3')

In [10]:
l()

KeyboardInterrupt: 